In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load data from CSV file
data = pd.read_csv("DL2.csv")

# Assuming the last column is the label, and rest are features
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

def create_model(neurons_1, neurons_2, optimizer):
    model = Sequential()
    model.add(Dense(neurons_1, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(neurons_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define different configurations
configurations = [
    (64, 64, 'adam'),
    (128, 128, 'adam'),
    (64, 128, 'adam'),
    (128, 64, 'adam'),
]

# Store results in a dictionary
results = {}

# Train models and store results
for i, (neurons_1, neurons_2, optimizer) in enumerate(configurations):
    model = create_model(neurons_1, neurons_2, optimizer)
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=0)
    results[f'Model_{i+1}'] = {
        'optimizer': optimizer,
        'train_accuracy': history.history['accuracy'][-1],
        'val_accuracy': history.history['val_accuracy'][-1],
        'train_loss': history.history['loss'][-1],
        'val_loss': history.history['val_loss'][-1],
    }

# Display results in a table
results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)

# Predict one value using the first model
sample_input = X_val[0].reshape(1, -1)
predicted_value = model.predict(sample_input)
print(f"Predicted Value for Sample Input: {predicted_value}")

        optimizer  train_accuracy  val_accuracy  train_loss    val_loss
Model_1      adam             0.0           0.0 -363.800171 -335.750580
Model_2      adam             0.0           0.0 -404.151123 -349.862701
Model_3      adam             0.0           0.0 -302.087708 -248.044632
Model_4      adam             0.0           0.0 -300.010834 -213.132080
1/1 [==============================] - 0s 88ms/step
Predicted Value for Sample Input: [[0.9999922]]


## exp5

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.optimizers import legacy

# Load the dataset
data = pd.read_csv('diabetes2.0.csv')

# Split features and target
X = data.drop('BloodPressure', axis=1)
y = data['BloodPressure']

# Splitting into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Define the architecture
architectures = [
    (64, 64),
    (128, 128),
    (64, 128),
    (128, 64)
]

# Define optimizer options
optimizers = [
    ('Adam', legacy.Adam()),
    ('SGD', legacy.SGD()),
    ('RMSprop', legacy.RMSprop())
]

# Dictionary to store results
results = {}

# Iterate over architectures and optimizers
for arch in architectures:
    for opt_name, opt in optimizers:
        model = Sequential()
        model.add(Dense(arch[0], activation='relu', input_shape=(X_train.shape[1],)))
        model.add(Dense(arch[1], activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

        history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_val_scaled, y_val), verbose=0)

        key = f"{arch}-{opt_name}"
        results[key] = {
            'train_accuracy': history.history['accuracy'],
            'val_accuracy': history.history['val_accuracy'],
            'train_loss': history.history['loss'],
            'val_loss': history.history['val_loss']
        }

# Print results
print("\t\tTrain Accuracy\tVal Accuracy\tTrain Loss\tVal Loss")
for key, value in results.items():
    print(key)
    epochs = len(value['train_accuracy'])
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}:\t{value['train_accuracy'][epoch]:.4f}\t\t{value['val_accuracy'][epoch]:.4f}\t\t{value['train_loss'][epoch]:.4f}\t\t{value['val_loss'][epoch]:.4f}")
    print()


		Train Accuracy	Val Accuracy	Train Loss	Val Loss
(64, 64)-Adam
Epoch 1:	0.0016		0.0000		-62.0116		-129.8361
Epoch 2:	0.0000		0.0000		-211.1181		-331.3201
Epoch 3:	0.0000		0.0000		-480.8654		-694.1302
Epoch 4:	0.0000		0.0000		-950.9773		-1301.4757
Epoch 5:	0.0000		0.0000		-1707.2314		-2260.8518
Epoch 6:	0.0000		0.0000		-2884.0215		-3693.3418
Epoch 7:	0.0000		0.0000		-4584.5737		-5746.3257
Epoch 8:	0.0000		0.0000		-6992.3022		-8568.0010
Epoch 9:	0.0000		0.0000		-10217.8662		-12266.5625
Epoch 10:	0.0000		0.0000		-14360.6855		-17060.4766

(64, 64)-SGD
Epoch 1:	0.0163		0.0714		nan		nan
Epoch 2:	0.0391		0.0714		nan		nan
Epoch 3:	0.0391		0.0714		nan		nan
Epoch 4:	0.0391		0.0714		nan		nan
Epoch 5:	0.0391		0.0714		nan		nan
Epoch 6:	0.0391		0.0714		nan		nan
Epoch 7:	0.0391		0.0714		nan		nan
Epoch 8:	0.0391		0.0714		nan		nan
Epoch 9:	0.0391		0.0714		nan		nan
Epoch 10:	0.0391		0.0714		nan		nan

(64, 64)-RMSprop
Epoch 1:	0.0016		0.0000		-102.4663		-210.7914
Epoch 2:	0.0000		0.0000		-329.8833		-475

In [ ]:
# Dictionary to store average validation accuracy
average_val_accuracy = {}

# Iterate over architectures and optimizers
for arch in architectures:
    for opt_name, _ in optimizers:
        key = f"{arch}-{opt_name}"
        average_val_accuracy[key] = np.mean(results[key]['val_accuracy'])

# Find the best parameters and approach
best_params_approach = max(average_val_accuracy, key=average_val_accuracy.get)
best_val_accuracy = average_val_accuracy[best_params_approach]

print(f"The best parameters and approach: {best_params_approach}")
print(f"Average Validation Accuracy: {best_val_accuracy:.4f}")


The best parameters and approach: (64, 64)-SGD
Average Validation Accuracy: 0.0714


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.base import BaseEstimator, ClassifierMixin

# Load the dataset
data = pd.read_csv('diabetes2.0.csv')

# Split features and target
X = data.drop('BloodPressure', axis=1)
y = data['BloodPressure']

# Splitting into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Function to create model
def create_model(arch1=64, arch2=64, optimizer='adam'):
    model = Sequential()
    model.add(Dense(arch1, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(arch2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define a wrapper class for the Keras model
class KerasClassifierWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, arch1=64, arch2=64, optimizer='adam', epochs=10, batch_size=32, verbose=0):
        self.arch1 = arch1
        self.arch2 = arch2
        self.optimizer = optimizer
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None

    def fit(self, X, y):
        self.model = create_model(arch1=self.arch1, arch2=self.arch2, optimizer=self.optimizer)
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        return self

    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype("int32")

# Define grid search parameters
architectures = {
    'arch1': [64, 128],
    'arch2': [64, 128],
}

optimizers = ['adam', 'sgd', 'rmsprop']

param_grid = dict(arch1=architectures['arch1'],
                  arch2=architectures['arch2'],
                  optimizer=optimizers)

# Perform grid search
model = KerasClassifierWrapper()
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Best: 0.039088 using {'arch1': 64, 'arch2': 64, 'optimizer': 'sgd'}
0.000000 (0.000000) with: {'arch1': 64, 'arch2': 64, 'optimizer': 'adam'}
0.039088 (0.000090) with: {'arch1': 64, 'arch2': 64, 'optimizer': 'sgd'}
0.000000 (0.000000) with: {'arch1': 64, 'arch2': 64, 'optimizer': 'rmsprop'}
0.000000 (0.000000) with: {'arch1': 64, 'arch2': 128, 'optimizer': 'adam'}
0.039088 (0.000090) with: {'arch1': 64, 'arch2': 128, 'optimizer': 'sgd'}
0.000000 (0.000000) with: {'arch1': 64, 'arch2': 128, 'optimizer': 'rmsprop'}
0.000000 (0.000000) with: {'arch1': 128, 'arch2': 64, 'optimizer': 'adam'}
0.039088 (0.000090) with: {'arch1': 128, 'arch2': 64, 'optimizer': 'sgd'}
0.000000 (0.000000) with: {'arch1': 128, 'arch2': 64, 'optimizer': 'rmsprop'}
0.000000 (0.000000) with: {'arch1': 128, 'arch2': 128, 'optimizer': 'adam'}
0.039088 (0.000090) with: {'arch1': 128, 'arch2': 128, 'optimizer': 'sgd'}
0.000000 (0.000000) with: {'arch1': 128, 'arch2': 128, 'optimizer': 'rmsprop'}
